---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
statesDict = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def lookup(string):
    statesDict = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    for acr,state in statesDict.items():
        if string == state:
            return acr

def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    df = pd.read_table('university_towns.txt',header=None)
    df.rename(columns={0:'RegionName'},inplace=True)
    statesList = []
    for i,val in df['RegionName'].iteritems():
        if '[edit]' in val:
            string = val.split('[')[0]
        statesList.append(string)
    df['State'] = statesList
    df = df[df['RegionName'].str.contains('edit') == False]
    df['RegionName'] = df['RegionName'].apply(lambda x: x.split('(')[0].rstrip() if '(' in x else x)
    df['State'] = df['State'].apply(lookup)
    return df.reset_index(drop=True)

In [4]:
get_list_of_university_towns()

RegionName State
0                          Auburn    AL
1                        Florence    AL
2                    Jacksonville    AL
3                      Livingston    AL
4                      Montevallo    AL
5                            Troy    AL
6                      Tuscaloosa    AL
7                        Tuskegee    AL
8                       Fairbanks    AK
9                       Flagstaff    AZ
10                          Tempe    AZ
11                         Tucson    AZ
12                    Arkadelphia    AR
13                         Conway    AR
14                   Fayetteville    AR
15                      Jonesboro    AR
16                       Magnolia    AR
17                     Monticello    AR
18                   Russellville    AR
19                         Searcy    AR
20                         Angwin    CA
21                         Arcata    CA
22                       Berkeley    CA
23                          Chico    CA
24                      Claremont    CA
25                         Cotati    CA
26                          Davis    CA
27                         Irvine    CA
28                     Isla Vista    CA
29   University Park, Los Angeles    CA
..                            ...   ...
487                          Wise    VA
488                    Chesapeake    VA
489                    Bellingham    WA
490                        Cheney    WA
491                    Ellensburg    WA
492                       Pullman    WA
493  University District, Seattle    WA
494                        Athens    WV
495                    Buckhannon    WV
496                      Fairmont    WV
497                     Glenville    WV
498                    Huntington    WV
499                    Montgomery    WV
500                    Morgantown    WV
501                 Shepherdstown    WV
502                  West Liberty    WV
503                      Appleton    WI
504                    Eau Claire    WI
505                     Green Bay    WI
506                     La Crosse    WI
507                       Madison    WI
508                     Menomonie    WI
509                     Milwaukee    WI
510                       Oshkosh    WI
511                   Platteville    WI
512                   River Falls    WI
513                 Stevens Point    WI
514                      Waukesha    WI
515                    Whitewater    WI
516                       Laramie    WY

[517 rows x 2 columns]

In [5]:
df_annual = pd.read_excel('gdplev.xls',skiprows=5,header=0,usecols='A:C')
df_annual.rename(columns={'Unnamed: 0': 'Year'},inplace=True)
df_annual.dropna(axis='columns',how='all',inplace=True)
df_annual.dropna(how='all',inplace=True)
df_annual['Year'] = df_annual['Year'].astype(int)
df_annual = df_annual[df_annual['Year']>=2000]
df_annual.reset_index(drop=True,inplace=True)
df_annual.set_index('Year',inplace=True)
df_annual.columns = pd.MultiIndex.from_product([['Annual'],['GDP in billions of current dollars',
                                                            'GDP in billions of chained 2009 dollars']])
df_annual

Annual  \
     GDP in billions of current dollars   
Year                                      
2000                            10284.8   
2001                            10621.8   
2002                            10977.5   
2003                            11510.7   
2004                            12274.9   
2005                            13093.7   
2006                            13855.9   
2007                            14477.6   
2008                            14718.6   
2009                            14418.7   
2010                            14964.4   
2011                            15517.9   
2012                            16155.3   
2013                            16691.5   
2014                            17393.1   
2015                            18036.6   

                                              
     GDP in billions of chained 2009 dollars  
Year                                          
2000                                 12559.7  
2001                                 12682.2  
2002                                 12908.8  
2003                                 13271.1  
2004                                 13773.5  
2005                                 14234.2  
2006                                 14613.8  
2007                                 14873.7  
2008                                 14830.4  
2009                                 14418.7  
2010                                 14783.8  
2011                                 15020.6  
2012                                 15354.6  
2013                                 15612.2  
2014                                 15982.3  
2015                                 16397.2

In [6]:
df_quarter = pd.read_excel('gdplev.xls',skiprows=5,header=0,usecols='E:G')
df_quarter.rename(columns={'Unnamed: 0': 'Quarter'},inplace=True)
df_quarter.dropna(axis='columns',how='all',inplace=True)
df_quarter.dropna(how='all',inplace=True)
df_quarter['Year'] = df_quarter['Quarter'].apply(lambda x: x.split('q')[0]).astype(int)
df_quarter = df_quarter[df_quarter['Year']>=2000]
df_quarter.reset_index(drop=True,inplace=True)
df_quarter.set_index(['Year','Quarter'],inplace=True)
df_quarter.columns = pd.MultiIndex.from_product([['Quarterly'],['GDP in billions of current dollars',
                                                            'GDP in billions of chained 2009 dollars']])

df = pd.merge(df_annual,df_quarter,how='inner',left_index=True,right_index=True)
df.reset_index(inplace=True)
df.set_index('Quarter',inplace=True)
gdp = 0
'''for i,val in df['Quarterly']['GDP in billions of chained 2009 dollars'].items():
        temp = val
        if val > gdp:
            gdp = val
            recession_end = i
recession_end
for i,val in df['Quarterly']['GDP in billions of chained 2009 dollars'].items():
        temp = val
        if val > gdp:
            gdp = val
            first = i
        elif val < gdp:
            second = i
            break
first'''
df['Quarterly']['GDP in billions of chained 2009 dollars'].idxmin()

'2000q1'

In [7]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdplev = pd.ExcelFile('gdplev.xls')
    gdplev = gdplev.parse("Sheet1", skiprows=219)
    gdplev = gdplev[['1999q4', 9926.1]]
    gdplev.columns = ['Quarter','GDP']
    for i in range(2, len(gdplev)):
        if (gdplev.iloc[i-2][1] > gdplev.iloc[i-1][1]) and (gdplev.iloc[i-1][1] > gdplev.iloc[i][1]):
            return gdplev.iloc[i-2][0]
get_recession_start()

'2008q3'

In [8]:
df_annual = pd.read_excel('gdplev.xls',skiprows=5,header=0)
df_annual = df_annual.iloc[:,:3]
df_annual.rename(columns={'Unnamed: 0': 'Year'},inplace=True)
df_annual.dropna(axis='columns',how='all',inplace=True)
df_annual.dropna(how='all',inplace=True)
df_annual['Year'] = df_annual['Year'].astype(int)
df_annual = df_annual[df_annual['Year']>=2000]
df_annual.reset_index(drop=True,inplace=True)
df_annual.set_index('Year',inplace=True)
df_annual.columns = pd.MultiIndex.from_product([['Annual'],['GDP in billions of current dollars',
                                                            'GDP in billions of chained 2009 dollars']])
df_quarter = pd.read_excel('gdplev.xls',skiprows=5,header=0)
df_quarter = df_quarter.iloc[:,4:]
df_quarter.rename(columns=lambda x: str(x).replace('.1',''))
df_quarter.rename(columns={'Unnamed: 4': 'Quarter'},inplace=True)
df_quarter.dropna(axis='columns',how='all',inplace=True)
df_quarter.dropna(how='all',inplace=True)
df_quarter['Year'] = df_quarter['Quarter'].apply(lambda x: x.split('q')[0]).astype(int)
df_quarter = df_quarter[df_quarter['Year']>=2000]
df_quarter.reset_index(drop=True,inplace=True)
df_quarter.set_index(['Year','Quarter'],inplace=True)
df_quarter.columns = pd.MultiIndex.from_product([['Quarterly'],['GDP in billions of current dollars',
                                                            'GDP in billions of chained 2009 dollars']])

df = pd.merge(df_annual,df_quarter,how='inner',left_index=True,right_index=True)
df.reset_index(inplace=True)
df.set_index('Quarter',inplace=True)
df = df['Quarterly']
df.reset_index(inplace=True)
gdp = 0
for i,val in df.iterrows():
    temp = val
    if (df.iloc[i-2][2] > df.iloc[i-1][2]) and (df.iloc[i-1][2] > df.iloc[i][2]):
        print(df.iloc[i-2][0])
        break

2008q2


In [68]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df_annual = pd.read_excel('gdplev.xls',skiprows=5,header=0)
    df_annual = df_annual.iloc[:,:3]
    df_annual.rename(columns={'Unnamed: 0': 'Year'},inplace=True)
    df_annual.dropna(axis='columns',how='all',inplace=True)
    df_annual.dropna(how='all',inplace=True)
    df_annual['Year'] = df_annual['Year'].astype(int)
    df_annual = df_annual[df_annual['Year']>=2000]
    df_annual.reset_index(drop=True,inplace=True)
    df_annual.set_index('Year',inplace=True)
    df_annual.columns = pd.MultiIndex.from_product([['Annual'],['GDP in billions of current dollars',
                                                                'GDP in billions of chained 2009 dollars']])
    df_quarter = pd.read_excel('gdplev.xls',skiprows=5,header=0)
    df_quarter = df_quarter.iloc[:,4:]
    df_quarter.rename(columns=lambda x: str(x).replace('.1',''))
    df_quarter.rename(columns={'Unnamed: 4': 'Quarter'},inplace=True)
    df_quarter.dropna(axis='columns',how='all',inplace=True)
    df_quarter.dropna(how='all',inplace=True)
    df_quarter['Year'] = df_quarter['Quarter'].apply(lambda x: x.split('q')[0]).astype(int)
    df_quarter = df_quarter[df_quarter['Year']>=2000]
    df_quarter.reset_index(drop=True,inplace=True)
    df_quarter.set_index(['Year','Quarter'],inplace=True)
    df_quarter.columns = pd.MultiIndex.from_product([['Quarterly'],['GDP in billions of current dollars',
                                                                'GDP in billions of chained 2009 dollars']])

    df = pd.merge(df_annual,df_quarter,how='inner',left_index=True,right_index=True)
    df.reset_index(inplace=True)
    df.set_index('Quarter',inplace=True)
    df = df['Quarterly']
    df.reset_index(inplace=True)
    for i,val in df.iterrows():
        if (df.iloc[i-2][1] > df.iloc[i-1][1]) and (df.iloc[i-1][1] > df.iloc[i][1]):
            return df.iloc[i-2][0]
get_recession_start()

'2008q3'

In [10]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    df_annual = pd.read_excel('gdplev.xls',skiprows=5,header=0)
    df_annual = df_annual.iloc[:,:3]
    df_annual.rename(columns={'Unnamed: 0': 'Year'},inplace=True)
    df_annual.dropna(axis='columns',how='all',inplace=True)
    df_annual.dropna(how='all',inplace=True)
    df_annual['Year'] = df_annual['Year'].astype(int)
    df_annual = df_annual[df_annual['Year']>=2000]
    df_annual.reset_index(drop=True,inplace=True)
    df_annual.set_index('Year',inplace=True)
    df_annual.columns = pd.MultiIndex.from_product([['Annual'],['GDP in billions of current dollars',
                                                                'GDP in billions of chained 2009 dollars']])
    df_quarter = pd.read_excel('gdplev.xls',skiprows=5,header=0)
    df_quarter = df_quarter.iloc[:,4:]
    df_quarter.rename(columns=lambda x: str(x).replace('.1',''))
    df_quarter.rename(columns={'Unnamed: 4': 'Quarter'},inplace=True)
    df_quarter.dropna(axis='columns',how='all',inplace=True)
    df_quarter.dropna(how='all',inplace=True)
    df_quarter['Year'] = df_quarter['Quarter'].apply(lambda x: x.split('q')[0]).astype(int)
    df_quarter = df_quarter[df_quarter['Year']>=2000]
    df_quarter.reset_index(drop=True,inplace=True)
    df_quarter.set_index(['Year','Quarter'],inplace=True)
    df_quarter.columns = pd.MultiIndex.from_product([['Quarterly'],['GDP in billions of current dollars',
                                                                'GDP in billions of chained 2009 dollars']])
    
    df = pd.merge(df_annual,df_quarter,how='inner',left_index=True,right_index=True)
    df.reset_index(inplace=True)
    df.set_index('Quarter',inplace=True)
    df = df['Quarterly']
    start = get_recession_start()
    df = df.loc[start:]
    df.reset_index(inplace=True)
    for i,val in df.iterrows():
        if (df.iloc[i-1][2] > df.iloc[i-2][2]) and (df.iloc[i][2] > df.iloc[i-1][2]):
            return df.iloc[i][0]

In [11]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    df_annual = pd.read_excel('gdplev.xls',skiprows=5,header=0)
    df_annual = df_annual.iloc[:,:3]
    df_annual.rename(columns={'Unnamed: 0': 'Year'},inplace=True)
    df_annual.dropna(axis='columns',how='all',inplace=True)
    df_annual.dropna(how='all',inplace=True)
    df_annual['Year'] = df_annual['Year'].astype(int)
    df_annual = df_annual[df_annual['Year']>=2000]
    df_annual.reset_index(drop=True,inplace=True)
    df_annual.set_index('Year',inplace=True)
    df_annual.columns = pd.MultiIndex.from_product([['Annual'],['GDP in billions of current dollars',
                                                                'GDP in billions of chained 2009 dollars']])
    df_quarter = pd.read_excel('gdplev.xls',skiprows=5,header=0)
    df_quarter = df_quarter.iloc[:,4:]
    df_quarter.rename(columns=lambda x: str(x).replace('.1',''))
    df_quarter.rename(columns={'Unnamed: 4': 'Quarter'},inplace=True)
    df_quarter.dropna(axis='columns',how='all',inplace=True)
    df_quarter.dropna(how='all',inplace=True)
    df_quarter['Year'] = df_quarter['Quarter'].apply(lambda x: x.split('q')[0]).astype(int)
    df_quarter = df_quarter[df_quarter['Year']>=2000]
    df_quarter.reset_index(drop=True,inplace=True)
    df_quarter.set_index(['Year','Quarter'],inplace=True)
    df_quarter.columns = pd.MultiIndex.from_product([['Quarterly'],['GDP in billions of current dollars',
                                                                'GDP in billions of chained 2009 dollars']])
    
    df = pd.merge(df_annual,df_quarter,how='inner',left_index=True,right_index=True)
    df.reset_index(inplace=True)
    df.set_index('Quarter',inplace=True)
    df = df['Quarterly']
    start = get_recession_start()
    end = get_recession_end()
    df = df.loc[start:end]
    return df['GDP in billions of chained 2009 dollars'].idxmin()

In [12]:
import math

In [13]:
def lookup(string):
    statesDict = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    if string in statesDict:
        return statesDict[string]
    return string

def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = pd.read_csv('City_Zhvi_AllHomes.csv')
    df['State'] = df['State'].apply(lookup)
    df = pd.merge(df.loc[:,:'SizeRank'],df.loc[:,'2000-01':],how='inner',left_index=True,right_index=True)
    df.set_index(['State','RegionName'],inplace=True)
    df = df.loc[:,'2000-01':]
    df.columns = df.columns.astype('datetime64')
    df = df.groupby(pd.PeriodIndex(df.columns,freq='Q'),axis=1).mean()
    df = df.rename(columns=lambda x: str(x).replace('Q','q'))
    return df

In [14]:
from scipy import stats

In [15]:
get_recession_start()

'2008q2'

In [146]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    university = get_list_of_university_towns()
    df = convert_housing_data_to_quarters()
    start = get_recession_start()
    end = get_recession_end()
    bottom = get_recession_bottom()
    df = df.loc[:,start:end]
    df.reset_index(inplace=True)
    df = pd.merge(df, university, how='left', left_on='RegionName',right_on='RegionName')
    df = df.rename(columns={'State_x':'State','State_y':'IsUni'})
    df['IsUni'] = df['IsUni'].fillna(0)
    df['IsUni'] = df['IsUni'].apply(lambda x: 1 if x is not 0 else 0)
    df = df.drop_duplicates().dropna()
    df['Price_Ratio'] = (df.loc[:,start]-df.loc[:,bottom])/df.loc[:,start]

    uni = df[df['IsUni']==1].loc[:,'Price_Ratio']
    non_uni = df[df['IsUni']==0].loc[:,'Price_Ratio']
    pval = stats.ttest_ind(uni, non_uni)[1]

    #Lower Mean Price Ratio
    if uni.mean() < non_uni.mean():
        better = 'university town'
    else:
        better = 'non-university town'
    different = pval<0.01
    result = (different,pval,better)
    return result

In [142]:
'''First creates new data showing the decline or growth of housing prices
between the recession start and the recession bottom. Then runs a ttest
comparing the university town values to the non-university towns values, 
return whether the alternative hypothesis (that the two groups are the same)
is true or not as well as the p-value of the confidence. 

Return the tuple (different, p, better) 
where 

different=True if the t-test is True at a p<0.01 (we reject the null hypothesis), 
different=False if otherwise (we cannot reject the null hypothesis). 

The variable p should
be equal to the exact p value returned from scipy.stats.ttest_ind(). The
value for better should be either "university town" or "non-university town"
depending on which has a lower mean price ratio (which is equivilent to a
reduced market loss).'''
data = convert_housing_data_to_quarters().copy()
data = data.loc[:,'2008q3':'2009q2']
data = data.reset_index()
def price_ratio(row):
    return (row['2008q3'] - row['2009q2'])/row['2008q3']

data['up&down'] = data.apply(price_ratio,axis=1)
#uni data 

uni_town = get_list_of_university_towns()['RegionName']
uni_town = set(uni_town)

def is_uni_town(row):
    #check if the town is a university towns or not.
    if row['RegionName'] in uni_town:
        return 1
    else:
        return 0
data['is_uni'] = data.apply(is_uni_town,axis=1)


not_uni = data[data['is_uni']==0].loc[:,'up&down'].dropna()
is_uni  = data[data['is_uni']==1].loc[:,'up&down'].dropna()
def better():
    if not_uni.mean() < is_uni.mean():
        return 'non-university town'
    else:
        return 'university town'
p_val = list(ttest_ind(not_uni, is_uni))[1]
result = (True,p_val,better())
result

(True, 0.00036641601595526971, 'university town')